<center><h1>CSCI 4140: Natural Language Processing</h1></center>
<center><h1>CSCI/DASC 6040: Computational Analysis of Natural Languages</h1></center>

<center><h6>Spring 2024</h6></center>
<center><h6>Homework 1 - N-gram models</h6></center>
<center><h6>Due Sunday, January 21, at 11:59 PM</h6></center>

<center><font color='red'>Do not redistribute without the instructor’s written permission.</font></center>

The learning goals of this assignment are to:

- Understand how to compute language model probabilities using maximum likelihood estimation.
- Implement back-off.
- Have fun using a language model to probabilistically generate texts.
- Compare word-level langauage models and character-level language models.

# Part 1: N-gram Language model (60 pts)

## Preliminaries

In [89]:
import random
from collections import *
import numpy as np

We'll start by loading the data. The WikiText language modeling dataset is a collection of tokens extracted from the set of verified Good and Featured articles on Wikipedia.

In [90]:
data = {'test': '', 'train': '', 'valid': ''}

for data_split in data:
    fname = "wiki.{}.tokens".format(data_split) # 데이터 분할을 기반으로 파일 이름 생성
    with open(fname, 'r') as f_wiki: # 해당 파일을 읽기 모드로 연다.
        data[data_split] = f_wiki.read().lower().split() # 파일을 읽어 소문자로 변환, 단어를 토큰으로 분할, data 딕셔너리에 저장

vocab = list(set(data['train'])) # train 데이터 분할에 대한 단어 목록 생성, set은 중복 제거

Now have a look at the data by running this cell.

In [91]:
print('train : %s ...' % data['train'][:10])
print('dev : %s ...' % data['valid'][:10])
print('test : %s ...' % data['test'][:10])
print('first 10 words in vocab: %s' % vocab[:10])

train : ['=', 'valkyria', 'chronicles', 'iii', '=', 'senjō', 'no', 'valkyria', '3', ':'] ...
dev : ['=', 'homarus', 'gammarus', '=', 'homarus', 'gammarus', ',', 'known', 'as', 'the'] ...
test : ['=', 'robert', '<unk>', '=', 'robert', '<unk>', 'is', 'an', 'english', 'film'] ...
first 10 words in vocab: ['groundbreaking', 'farce', 'proteomics', 'deficits', 'missy', 'libretto', 'sacristy', 'bc2', 'flap', '18-']


## Q1.1: Train N-gram language model (25 pts)

Complete the following `train_ngram_lm` function based on the following input/output specifications. If you've done it right, you should pass the tests in the cell below.

*Input:*
+ **data**: the data object created in the cell above that holds the tokenized Wikitext data
+ **order**: the order of the model (i.e., the "n" in "n-gram" model). If order=3, we compute $p(w_2 | w_0, w_1)$.

*Output:*
+ **lm**: A dictionary where the key is the history and the value is a probability distribution over the next word computed using the maximum likelihood estimate from the training data. Importantly, this dictionary should include *backoff* probabilities as well; e.g., for order=4, we want to store $p(w_3 | w_0,w_1,w_2)$ as well as $p(w_3|w_1,w_2)$ and $p(w_3|w_2)$. 

Each key should be a single string where the words that form the history have been concatenated using spaces. Given a key, its corresponding value should be a dictionary where each word type in the vocabulary is associated with its probability of appearing after the key. For example, the entry for the history 'w1 w2' should look like:

    
    lm['w1 w2'] = {'w0': 0.001, 'w1' : 1e-6, 'w2' : 1e-6, 'w3': 0.003, ...}
    
In this example, we also want to store `lm['w2']` and `lm['']`, which contain the bigram and unigram distributions respectively.

*Hint:* You might find the **defaultdict** and **Counter** classes in the **collections** module to be helpful.

In [92]:
def train_ngram_lm(data, order=3):
    """
        Train n-gram language model
    """
    
    # pad (order-1) special tokens to the left
    # for the first token in the text
    order -= 1
    data = ['<S>'] * order + data
    lm = defaultdict(Counter)
    for i in range(len(data) - order):
        '''
        IMPLEMENT HERE !
        '''
        for o in range(0, order+1):  # 0부터 order까지의 크기의 히스토리에 대해 반복하기
            history = ' '.join(data[i:i+o])
            lm[history][data[i+o]] += 1

    # Normalize probabilities for each history
    for history, word_counts in lm.items():
        total_count = sum(word_counts.values())
        for word in word_counts:
            word_counts[word] /= total_count

    return lm   

In [93]:
def test_ngram_lm():
  
    print('checking empty history ...')
    lm1 = train_ngram_lm(data['train'], order=1)
    assert '' in lm1, "empty history should be in the language model!"
    
    print('checking probability distributions ...')
    lm2 = train_ngram_lm(data['train'], order=2)
    sample = [sum(lm2[k].values()) for k in random.sample(list(lm2), 10)]
    assert all([a > 0.999 and a < 1.001 for a in sample]), "lm[history][word] should sum to 1!"
    
    print('checking lengths of histories ...')
    lm3 = train_ngram_lm(data['train'], order=3)
    assert len(set([len(k.split()) for k in list(lm3)])) == 3, "lm object should store histories of all sizes!"
    
    print('checking word distribution values ...')
    assert lm1['']['the'] < 0.064 and lm1['']['the'] > 0.062 and \
           lm2['the']['first'] < 0.017 and lm2['the']['first'] > 0.016 and \
           lm3['the first']['time'] < 0.106 and lm3['the first']['time'] > 0.105, \
           "values do not match!"
    
    print("Congratulations, you passed the ngram check!")
    
test_ngram_lm()

checking empty history ...
checking probability distributions ...
checking lengths of histories ...
checking word distribution values ...
Congratulations, you passed the ngram check!


## Q1.2: Generate text from n-gram language model (10 pts)

Complete the following `generate_text` function based on these input/output requirements:

*Input:*

+ **lm**: the lm object is the dictionary you return from  the **train_ngram_lm** function
+ **vocab**: vocab is a list of unique word types in the training set, already computed for you during data loading.
+ **context**: the input context string that you want to condition your language model on, should be a space-separated string of tokens
+ **order**: order of your language model (i.e., "n" in the "n-gram" model)
+ **num_tok**: number of tokens to be generated following the input context


*Output:*

+ generated text, should be a space-separated string
    
*Hint:*

After getting the next-word distribution given history, try using **[numpy.random.choice](https://numpy.org/doc/stable/reference/random/generated/numpy.random.choice.html)** to sample the next word from the distribution.

In [94]:
def generate_text(lm, vocab, context="he is the", order=3, num_tok=25):
    # The goal is to generate new words following the context
    # If context has more tokens than the order of lm, 
    # generate text that follows the last (order-1) tokens of the context
    # and store it in the variable ⁠ history ⁠
    order -= 1
    history = ' '.join(context.split()[-order:])
    
    # ⁠ out ⁠ is the list of tokens of context
    # you need to append the generated tokens to this list
    out = context.split()

    for i in range(num_tok):  # generate num_tok tokens
        if history not in lm:
            break  # Break if history not found in language model

        # Get the next-word distribution given history
        next_word_probs = lm[history]

        # Sample the next word from the distribution
        next_word = np.random.choice(list(next_word_probs.keys()), p=list(next_word_probs.values()))

        # Append the generated token to the list
        out.append(next_word)

        # Update history for the next iteration
        history = ' '.join(out[-order:])

    return ' '.join(out)

In [95]:
order = 1
generate_text(train_ngram_lm(data['train'], order=order), vocab, context='he is the', order=order)

'he is the'

Now try to generate some texts, generated by ngram language model with different orders.

In [96]:
order = 1
generate_text(train_ngram_lm(data['train'], order=order), vocab, context='he is the', order=order)

'he is the'

In [97]:
order = 2
generate_text(train_ngram_lm(data['train'], order=order), vocab, context='he is the', order=order)

'he is the 1950s , " . = = 7 million , the war ii granted 200 homes were examining capitalism ) = = = " , s.'

In [98]:
order = 3
generate_text(train_ngram_lm(data['train'], order=order), vocab, context='he is the', order=order)

'he is the setting of sean paul on " bassline " garnered four major " , not the little rock under the names of minor pieces . to'

In [99]:
order = 4
generate_text(train_ngram_lm(data['train'], order=order), vocab, context='he is the', order=order)

'he is the managing director of clean edge , a research and strategy firm in the south to bay city along the m @-@ 6 crosses over division'

## Q1.3 : Evaluate the models (25 pts)
Now let's evaluate the models quantitively using the intrinsic metric **perplexity**. 

Recall perplexity is the inverse probability of the test text
$$\text{PP}(w_1, \dots, w_t) = P(w_1, \dots, w_t)^{-\frac{1}{T}}$$

For an n-gram model, perplexity is computed by
$$\text{PP}(w_1, \dots, w_t) = \left[\prod_{t=1}^T P(w_t|w_{t-1},\ldots,w_{t-n+1})\right]^{-\frac{1}{T}}$$

To address the numerical issue (underflow), we usually compute
$$\text{PP}(w_1, \dots, w_t) = \exp\left(-\frac{1}{T}\sum_i \log P(w_t|w_{t-1},\ldots,w_{t-n+1})\right)$$


*Input:*

+ **lm**: the language model you trained (the object you returned from the `train_ngram_lm` function)
+ **data**: test data
+ **vocab**: the list of unique word types in the training set
+ **order**: order of the lm

*Output:*

+ the perplexity of test data

*Hint:*

+ If the history is not in the **lm** object, back-off to (n-1) order history to check if it is in **lm**. If no history can be found, just use `1/|V|` where `|V|` is the size of vocabulary.

In [163]:
from math import log, exp
def compute_perplexity(lm, data, vocab, order=3):
    
    # pad according to order
    order -= 1
    data = ['<S>'] * order + data
    log_sum = 0

    for i in range(len(data) - order):
        h, w = ' '.join(data[i: i+order]), data[i+order]
        """
        IMPLEMENT ME!
        """
        
        current_order = order  # Preserve the original order value
        
        # if h not in lm, back-off to n-1 gram and look up again
        while h not in lm and current_order > 1:
            current_order -= 1
            h = ' '.join(data[i: i+current_order])
        
        if h in lm:
            # Get the distribution for the next word
            next_word_dist = lm[h]
            
            # Calculate the conditional probability
            conditional_prob = next_word_dist.get(w, 1/len(vocab))  # Use 1/|V| if w not in lm[h]
            
            # Accumulate the log probability
            log_sum += log(conditional_prob)
        else:
            # Handle the case where h is not in lm even after back-off
            # Use 1/|V| as a default probability
            log_sum += log(1/len(vocab))
    
    # Calculate perplexity
    perplexity = exp(-log_sum / (len(data) - (order - 1)))
    
    return perplexity

Let's evaluate the language model with different orders. You should see a decrease in perplexity as the order increases. As a reference, the perplexity of the unigram, bigram, trigram, and 4-gram language models should be around 795, 203, 141, and 130 respectively.

In [164]:
for o in [1, 2, 3, 4]:
    lm = train_ngram_lm(data['train'], order=o)
    print('order {} ppl {}'.format(o, compute_perplexity(lm, data['test'], vocab, order=o)))

order 1 ppl 24.499292666887136
order 2 ppl 10.368212096425294
order 3 ppl 59.27749291482931
order 4 ppl 92.59853413144387


# Part 2: Character-level N-gram language model (50 points)

In the lecture, language modeling was defined as the task of predicting the next word in a sequence given the previous words. In this part of the assignment, we will focus on the related problem of predicting the next character or word in a sequence given the previous characters.

## Preliminaries

We have to modify how we load the data, by splitting it into characters rather than words. Also, we'll use both upper case and lower case letters.

In [102]:
data = {'test': '', 'train': '', 'valid': ''}

for data_split in data:
    fname = "wiki.{}.tokens".format(data_split)
    data[data_split] = list(open(fname, 'r', encoding = 'utf-8').read())

vocab = list(set(data['train']))

Now have a look at the data by running this cell.

In [103]:
print('train : %s ...' % data['train'][:10])
print('dev : %s ...' % data['valid'][:10])
print('test : %s ...' % data['test'][:10])
print('first 10 characters in vocab: %s' % vocab[:10])

train : [' ', '\n', ' ', '=', ' ', 'V', 'a', 'l', 'k', 'y'] ...
dev : [' ', '\n', ' ', '=', ' ', 'H', 'o', 'm', 'a', 'r'] ...
test : [' ', '\n', ' ', '=', ' ', 'R', 'o', 'b', 'e', 'r'] ...
first 10 characters in vocab: ['p', 'q', 'リ', 'ვ', 'v', 'R', 't', 'α', 'ớ', '′']


## Q2.1: Train N-gram language model (20 pts)

Complete the following `train_ngram_lm` function based on the following input/output specifications. If you've done it right, you should pass the tests in the cell below.

*Input:*
+ **data**: the data object created in the cell above that holds the tokenized Wikitext data
+ **order**: the order of the model (i.e., the "n" in "n-gram" model). If order=3, we compute $p(c_2 | c_0, c_1)$.

*Output:*
+ **lm**: A dictionary where the key is the history and the value is a probability distribution over the next character computed using the maximum likelihood estimate from the training data. Importantly, this dictionary should include *backoff* probabilities as well; e.g., for order=4, we want to store $p(c_3 | c_0,c_1,c_2)$ as well as $p(c_3|c_1,c_2)$ and $p(c_3|c_2)$. 

Each key should be a single string where the characters that form the history have been concatenated. Given a key, its corresponding value should be a dictionary where each character in the vocabulary is associated with its probability of appearing after the key. For example, the entry for the history 'c1c2' should look like:

    
    lm['c1c2'] = {'c0': 0.001, 'c1' : 1e-6, 'c2' : 1e-6, 'c3': 0.003, ...}
    
In this example, we also want to store `lm['c2']` and `lm['']`, which contain the bigram and unigram distributions respectively.

*Hint:* You might find the **defaultdict** and **Counter** classes in the **collections** module to be helpful.

In [104]:
def train_ngram_lm(data, order=3):
    """
        Train n-gram language model
    """
    
    # pad (order-1) special tokens to the left
    # for the first token in the text
    order -= 1
    data = ['~'] * order + data # 
    
    lm = defaultdict(Counter)
    
    for i in range(len(data) - order):
        for o in range(0, order+1): # loops through different variations of ngrams
            history = ''.join(data[i:i+o]) # concatinates words as described above, word at index i+o exclusive
            lm[history][data[i+o]] += 1 # update counter for word aqt index i+1 (aka next word in sequence)

    for history in lm:
        # normalization
        total_count = sum(lm[history].values()) # sum up all counts for a particular history
        lm[history] = {w: cnt / total_count for w, cnt in lm[history].items()} # create dict as described above: for each word in history, it becomes the key, and the distribution of the next word becomes the value
    return dict(lm)

In [105]:
def test_ngram_lm():
  
    print('checking empty history ...')
    lm1 = train_ngram_lm(data['train'], order=1)
    assert '' in lm1, "empty history should be in the language model!"
    
    print('checking probability distributions ...')
    lm2 = train_ngram_lm(data['train'], order=2)
    sample = [sum(lm2[k].values()) for k in random.sample(list(lm2), 10)]
    assert all([a > 0.999 and a < 1.001 for a in sample]), "lm[history][character] should sum to 1!"
    
    print('checking lengths of histories ...')
    lm3 = train_ngram_lm(data['train'], order=3)
    assert len(set([len(k) for k in list(lm3)])) == 3, "lm object should store histories of all sizes!"
    
    print('checking character distribution values ...')
    assert lm1['']['t'] < 0.062 and lm1['']['t'] > 0.060 and \
           lm2['t']['h'] < 0.297 and lm2['t']['h'] > 0.296 and \
           lm3['th']['e'] < 0.694 and lm3['th']['e'] > 0.693, \
           "values do not match!"
    
    print("Congratulations, you passed the ngram check!")
    
test_ngram_lm()

checking empty history ...
checking probability distributions ...
checking lengths of histories ...
checking character distribution values ...
Congratulations, you passed the ngram check!


## Q2.2: Generate text from n-gram language model (10 pts)

Complete the following `generate_text` function based on these input/output requirements:

*Input:*

+ **lm**: the lm object is the dictionary you return from  the **train_ngram_lm** function
+ **vocab**: vocab is a list of unique characters in the training set, already computed for you during data loading.
+ **context**: the input context string that you want to condition your language model on, should be a string
+ **order**: order of your language model (i.e., "n" in the "n-gram" model)
+ **num_tok**: number of characters to be generated following the input context


*Output:*

+ generated text, should be a sequence of characters
    
*Hint:*

After getting the next-character distribution given history, try using **[numpy.random.choice](https://numpy.org/doc/stable/reference/random/generated/numpy.random.choice.html)** to sample the next character from the distribution.

In [125]:
# generate text
def generate_text(lm, vocab, context="he ", order=3, num_tok=25):
    
    # The goal is to generate new characters following the context
    # If context has more tokens than the order of lm, 
    # generate text that follows the last (order-1) tokens of the context
    # and store it in the variable `history`
    order -= 1
    history = list(context)[-order:]
    # `out` is the list of tokens of context
    # you need to append the generated tokens to this list
    out = list(context)
        
    for i in range(num_tok):
        # Get the history string to use as the key for lm
        history_str = ''.join(history)
        
        # Check if history_str is in the lm
        if history_str in lm:
            # Get the distribution for the next character
            next_char_dist = lm[history_str]
            
            # Make sure vocab and probabilities have the same size
            vocab_subset = [char for char in vocab if char in next_char_dist]
            probabilities = [next_char_dist[char] for char in vocab_subset]
            
            # Sample the next character from the distribution
            next_char = np.random.choice(vocab_subset, p=probabilities)
            
            # Append the sampled character to the output
            out.append(next_char)
            
            # Update the history by removing the first character and adding the sampled character
            history = history[1:] + [next_char]
        else:
            # If history_str is not in lm, break the loop
            break
    
    # Convert the output list to a string
    generated_text = ''.join(out)
    
    return generated_text

Now try to generate some texts, generated by ngram language model with different orders.

In [126]:
order = 1
generate_text(train_ngram_lm(data['train'], order=order), vocab, context='he is the', order=order)

'he is the'

In [127]:
order = 2
generate_text(train_ngram_lm(data['train'], order=order), vocab, context='he is the', order=order)

"he is the t , 's Ulthaff sthambe f"

In [128]:
order = 3
generate_text(train_ngram_lm(data['train'], order=order), vocab, context='he is the', order=order)

'he is thered tubish hac , thervals'

In [129]:
order = 4
generate_text(train_ngram_lm(data['train'], order=order), vocab, context='he is the', order=order)

'he is their work up the goes " wer'

## Q2.3 : Evaluate the models (20 pts)
Now let's evaluate the models quantitively using the intrinsic metric **perplexity**. 

Recall perplexity is the inverse probability of the test text
$$\text{PP}(w_1, \dots, w_t) = P(w_1, \dots, w_t)^{-\frac{1}{T}}$$

For an n-gram model, perplexity is computed by
$$\text{PP}(w_1, \dots, w_t) = \left[\prod_{t=1}^T P(w_t|w_{t-1},\ldots,w_{t-n+1})\right]^{-\frac{1}{T}}$$

To address the numerical issue (underflow), we usually compute
$$\text{PP}(w_1, \dots, w_t) = \exp\left(-\frac{1}{T}\sum_i \log P(w_t|w_{t-1},\ldots,w_{t-n+1})\right)$$


*Input:*

+ **lm**: the language model you trained (the object you returned from the `train_ngram_lm` function)
+ **data**: test data
+ **vocab**: the list of unique characters in the training set
+ **order**: order of the lm

*Output:*

+ the perplexity of test data

*Hint:*

+ If the history is not in the **lm** object, back-off to (n-1) order history to check if it is in **lm**. If no history can be found, just use `1/|V|` where `|V|` is the size of vocabulary.

In [154]:
from math import log, exp
def compute_perplexity(lm, data, vocab, order=3):
    
    # pad according to order
    order -= 1
    data = ['~'] * order + data
    log_sum = 0
    for i in range(len(data) - order):
        h, w = ''.join(data[i: i+order]), data[i+order]
        """
        IMPLEMENT ME!
        # if h not in lm, back-off to n-1 gram and look up again
        """
        current_order = order  # Preserve the original order value
        
        # if h not in lm, back-off to n-1 gram and look up again
        while h not in lm and current_order > 1:
            current_order -= 1
            h = ' '.join(data[i: i+current_order])
        
        if h in lm:
            # Get the distribution for the next word
            next_word_dist = lm[h]
            
            # Calculate the conditional probability
            conditional_prob = next_word_dist.get(w, 1/len(vocab))  # Use 1/|V| if w not in lm[h]
            
            # Accumulate the log probability
            log_sum += log(conditional_prob)
        else:
            # Handle the case where h is not in lm even after back-off
            # Use 1/|V| as a default probability
            log_sum += log(1/len(vocab))
    
    # Calculate perplexity
    perplexity = exp(-log_sum / (len(data) - (order - 1)))
    
    return perplexity

Let's evaluate the language model with different orders. You should see a decrease in perplexity as the order increases. As a reference, the perplexity of the unigram, bigram, trigram, and 4-gram language models should be around 24.5, 10.4, 6.5, and 4.5 respectively.

In [151]:
for o in [1, 2, 3, 4]:
    lm = train_ngram_lm(data['train'], order=o)
    print('order {} ppl {}'.format(o, compute_perplexity(lm, data['test'], vocab, order=o)))

order 1 ppl 24.499292666887136
order 2 ppl 10.36816545724361
order 3 ppl 6.511296801517689
order 4 ppl 4.462984318065475
